In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd

User parameters


In [ ]:
nb_samples_by_batch = 10000

quantity_of_interest = ["mean", "variance"]

Samples (sobol sequence)


In [ ]:
input_parameters = pd.read_parquet("./input_samples.parquet")
input_parameters

Seting the output file


In [ ]:
output_sobol_index_filepath = Path("./output_sobol_index.parquet")
if output_sobol_index_filepath.exists():
    output_sobol_index = pd.read_parquet(output_sobol_index_filepath)
else:
    output_sobol_index = pd.DataFrame(columns=quantity_of_interest)
    output_sobol_index.to_parquet(output_sobol_index_filepath)
output_sobol_index

---

Cost function definition


In [ ]:
def batch_cost_function_execution(input_parameters: pd.DataFrame) -> np.ndarray:
    return np.full((input_parameters.shape[0], len(quantity_of_interest)), 1)

---


Run as much batch you can


In [ ]:
for batch_number in range(0, (len(input_parameters) // nb_samples_by_batch) + 1):
    min_batch = batch_number * nb_samples_by_batch
    max_batch = min(batch_number * nb_samples_by_batch + nb_samples_by_batch, len(input_parameters))
    print(f"Batch {batch_number} = {min_batch} : {max_batch}")

    if not (max_batch) in output_sobol_index.index:
        batch_samples = input_parameters.iloc[
            batch_number * nb_samples_by_batch : batch_number * nb_samples_by_batch + nb_samples_by_batch
        ]

        results = batch_cost_function_execution(batch_samples)

        output_sobol_index = pd.concat(
            [output_sobol_index, pd.DataFrame(results, columns=quantity_of_interest, index=batch_samples.index)]
        )
        output_sobol_index.to_parquet(output_sobol_index_filepath)

Show output


In [ ]:
output_sobol_index